# svtk for 4 KOLF2.1 cell lines

### Nov 10th, 2020

Last time I followed the script from Anni.
This time try to use the procedures from the github of Dr. Talkowski lab 

This the comments from Anni "I’ve attached my script and I believe all the necessary external files but let me know if I forgot any. Basically in the script I ran manta version 1.6 to call the variants but had to convert the output back to 1.4 versioning since the newer versioning didn’t handle inversions correctly from what I remember. Then following that I was able to filter and standardize".

These are comments from Kim "For our current PD analysis we actually ran an **older version** of manta with default settings, we are then running gnomad-SV with the manta output that runs multiple filtering modules within the pipeline."




In [2]:
%cd /Users/pengl7/Downloads/sv

/Users/pengl7/Downloads/WGS/SV_calling


## copy thee files to local to do inversion reformat

In [114]:
!gsutil ls {PROJECT_BUCKET}/{COHORT}/sv-manta/

gs://test-7cee72c0e768/Jax/sv-manta/KOLF2-ARID2-A2/
gs://test-7cee72c0e768/Jax/sv-manta/KUCG3-C1/
gs://test-7cee72c0e768/Jax/sv-manta/LNGPI1-C1/
gs://test-7cee72c0e768/Jax/sv-manta/NCRM1-C6/
gs://test-7cee72c0e768/Jax/sv-manta/NCRM5-C5/
gs://test-7cee72c0e768/Jax/sv-manta/NN0003932-C3/
gs://test-7cee72c0e768/Jax/sv-manta/NN0004297-C1/
gs://test-7cee72c0e768/Jax/sv-manta/PGP1-C2/


In [129]:
%%bash -s $PROJECT_BUCKET $COHORT

PROJECT_BUCKET=${1}
COHORT=${2}

for SAMPLE in 'KOLF2-ARID2-A2' 'KUCG3-C1' 'LNGPI1-C1' 'NCRM1-C6' 'NCRM5-C5' 'NN0003932-C3' 'NN0004297-C1' 'PGP1-C2'
do 
gsutil cp -r ${PROJECT_BUCKET}/${COHORT}/sv-manta/${SAMPLE}/${SAMPLE}.diploidSV.vcf.gz* manta-output/
done

Copying gs://test-7cee72c0e768/Jax/sv-manta/KOLF2-ARID2-A2/KOLF2-ARID2-A2.diploidSV.vcf.gz...
Copying gs://test-7cee72c0e768/Jax/sv-manta/KOLF2-ARID2-A2/KOLF2-ARID2-A2.diploidSV.vcf.gz.tbi...
/ [2 files][856.9 KiB/856.9 KiB]                                                
Operation completed over 2 objects/856.9 KiB.                                    
Copying gs://test-7cee72c0e768/Jax/sv-manta/KUCG3-C1/KUCG3-C1.diploidSV.vcf.gz...
Copying gs://test-7cee72c0e768/Jax/sv-manta/KUCG3-C1/KUCG3-C1.diploidSV.vcf.gz.tbi...
- [2 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 2 objects/1.0 MiB.                                      
Copying gs://test-7cee72c0e768/Jax/sv-manta/LNGPI1-C1/LNGPI1-C1.diploidSV.vcf.gz...
Copying gs://test-7cee72c0e768/Jax/sv-manta/LNGPI1-C1/LNGPI1-C1.diploidSV.vcf.gz.tbi...
- [2 files][769.5 KiB/769.5 KiB]                                                
Operation completed over 2 objects/769.5 KiB.                

In [133]:
%%bash
for SAMPLE in 'KOLF2-ARID2-A2' 'KUCG3-C1' 'LNGPI1-C1' 'NCRM1-C6' 'NCRM5-C5' 'NN0003932-C3' 'NN0004297-C1' 'PGP1-C2'
do
gunzip manta-output/${SAMPLE}.diploidSV.vcf.gz
done 

In [134]:
%ls -lth manta-output/

total 52216
-rw-r--r--  1 pengl7  1360859114    90K Sep  9 09:43 PGP1-C2.diploidSV.vcf.gz.tbi
-rw-r--r--  1 pengl7  1360859114   3.1M Sep  9 09:43 PGP1-C2.diploidSV.vcf
-rw-r--r--  1 pengl7  1360859114    94K Sep  9 09:43 NN0004297-C1.diploidSV.vcf.gz.tbi
-rw-r--r--  1 pengl7  1360859114   3.1M Sep  9 09:43 NN0004297-C1.diploidSV.vcf
-rw-r--r--  1 pengl7  1360859114    85K Sep  9 09:43 NN0003932-C3.diploidSV.vcf.gz.tbi
-rw-r--r--  1 pengl7  1360859114   2.7M Sep  9 09:43 NN0003932-C3.diploidSV.vcf
-rw-r--r--  1 pengl7  1360859114    88K Sep  9 09:43 NCRM5-C5.diploidSV.vcf.gz.tbi
-rw-r--r--  1 pengl7  1360859114   2.9M Sep  9 09:43 NCRM5-C5.diploidSV.vcf
-rw-r--r--  1 pengl7  1360859114    91K Sep  9 09:43 NCRM1-C6.diploidSV.vcf.gz.tbi
-rw-r--r--  1 pengl7  1360859114   3.1M Sep  9 09:43 NCRM1-C6.diploidSV.vcf
-rw-r--r--  1 pengl7  1360859114    87K Sep  9 09:43 LNGPI1-C1.diploidSV.vcf.gz.tbi
-rw-r--r--  1 pengl7  1360859114   2.8M Sep  9 09:43 LNGPI1-C1.diploidSV.vcf
-rw-r--r--  1 peng

## Prepare the needed files

copy the conversion.py file to to manta-out folde
download the reference fasta
install samtools


In [135]:
!gsutil -m cp gs://gcp-public-data--broad-references/hg38/v0/Homo_sapiens_assembly38.fasta manta-output/

Copying gs://gcp-public-data--broad-references/hg38/v0/Homo_sapiens_assembly38.fasta...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").


Operation completed over 1 objects/3.0 GiB.                                      


### Samtools installed using anaconda in my mac doesn't work

dyld: Library not loaded: @rpath/libcrypto.1.0.0.dylib
  Referenced from: /Users/pengl7/opt/anaconda3/envs/genomics/bin/samtools
  Reason: image not found
Abort trap: 6



In [ ]:
# since my samtools installed using anaconda doesn't work in my local mac, do the following in biowulf
# actaully I still can run the following cmds in local mac becasue I have installed samtools through the c compile method.
# exeute the following commands in terminal not in jupyter notebook
module load python/2.7 samtools
for sample in 'KOLF2-ARID2-A2' 'KUCG3-C1' 'LNGPI1-C1' 'NCRM1-C6' 'NCRM5-C5' 'NN0003932-C3' 'NN0004297-C1' 'PGP1-C2'
do
python convertInversion.py /usr/local/Anaconda/envs/py2.7/bin/samtools Homo_sapiens_assembly38.fasta ${sample}.diploidSV.vcf > ${sample}.diploidSV.1.4.vcf
done 

## Solutions for samtools in local mac

conda uninstall samtools
cp the samtools executable into /usr/local/bin/

In [152]:
!which samtools

/usr/local/bin/samtools


##  Two files failed due to the following error (NCRM1-C6.diploidSV.vcf and LNGPI1-C1.diploidSV.vcf)
```Traceback (most recent call last):
  File "convertInversion.py", line 291, in <module>
    invMateDict = scanVcf(vcfFile)
  File "convertInversion.py", line 108, in scanVcf
    vcfRec.checkInversion()
  File "convertInversion.py", line 73, in checkInversion
    getMateInfo(']')
  File "convertInversion.py", line 64, in getMateInfo
[self.mateChr, matePos] = items[1].split(':')```


## Solutions for convertInversions

Filter the vcf using ":PASS:" or "PASS"

In [174]:
##used Mantas convertInversion.py script to covert version 1.6 output back to 1.4 to deal w inversion (ran on local mac)

print('ran in terminal, python 2 environment of local ((needs to run with python2!))')
print('')
print("cd manta-output/")
print("")
for sample in ['LNGPI1-C1', 'NCRM1-C6']:
    # using (grep "^#" {sample}.diploidSV.vcf; awk '$7 ~ "PASS"' {sample}.diploidSV.vcf) > is better
    cmd1=f'(grep "^#" {sample}.diploidSV.vcf; grep ":PASS:" {sample}.diploidSV.vcf) > {sample}.diploidSV.pass.vcf'
    print(cmd1)
    cmd = f'python2.7 convertInversion.py /usr/local/bin/samtools Homo_sapiens_assembly38.fasta {sample}.diploidSV.pass.vcf > {sample}.diploidSV.1.4.vcf'
    print(cmd)
    print()
    

ran in terminal, python 2 environment of local ((needs to run with python2!))

cd manta-output/

(grep "^#" LNGPI1-C1.diploidSV.vcf; grep ":PASS:" LNGPI1-C1.diploidSV.vcf) > LNGPI1-C1.diploidSV.pass.vcf
python2.7 convertInversion.py /usr/local/bin/samtools Homo_sapiens_assembly38.fasta LNGPI1-C1.diploidSV.pass.vcf > LNGPI1-C1.diploidSV.1.4.vcf

(grep "^#" NCRM1-C6.diploidSV.vcf; grep ":PASS:" NCRM1-C6.diploidSV.vcf) > NCRM1-C6.diploidSV.pass.vcf
python2.7 convertInversion.py /usr/local/bin/samtools Homo_sapiens_assembly38.fasta NCRM1-C6.diploidSV.pass.vcf > NCRM1-C6.diploidSV.1.4.vcf



## Run SVTK 

For SVTK, https://github.com/talkowski-lab/svtk, here are the major steps to run locally:

In [169]:
print(WRKDIR)
print(WRKDIR2)

/Users/pengl7/Downloads/WGS/SV_calling
/Users/pengl7/Downloads/WGS/SV_calling/manta-output


## The two blocks below are different from Anni's syntax, but resutls are the same
- Block 1: only select the calls PASS filter
- Block 2:  change format: get rid of "chr" prefix

In [ ]:
# go to the folder of manta-output in local mac
# excute the following commands; clean Manta output: 1.select "PASS" calls 
for sample in 'KOLF2-ARID2-A2' 'KUCG3-C1' 'LNGPI1-C1' 'NCRM1-C6' 'NCRM5-C5' 'NN0003932-C3' 'NN0004297-C1' 'PGP1-C2'
do
(grep "^#" ${sample}.diploidSV.1.4.vcf; awk '$7 == "PASS"' ${sample}.diploidSV.1.4.vcf) >  temp.${sample}.diploidSV.1.4.vcf
done 

In [ ]:
# run in local terminal; replace chr with nothing
# gsub(regex, sub, string). gsub stands for global substitution. It replaces every occurrence of regex with the given string (sub). The third parameter is optional.
# in the folder of manta-output in local mac
for sample in 'KOLF2-ARID2-A2' 'KUCG3-C1' 'LNGPI1-C1' 'NCRM1-C6' 'NCRM5-C5' 'NN0003932-C3' 'NN0004297-C1' 'PGP1-C2'
do
awk '{{ gsub(/chr/, ""); print}}' temp.${sample}.diploidSV.1.4.vcf > ${sample}.diploidSV.1.4.edit.vcf
done 

In [ ]:
# run in local terminal
# #checking for inversions -> found none in any files
for sample in 'KOLF2-ARID2-A2' 'KUCG3-C1' 'LNGPI1-C1' 'NCRM1-C6' 'NCRM5-C5' 'NN0003932-C3' 'NN0004297-C1' 'PGP1-C2'
do
awk '$5 == "<INV>"' ${sample}.diploidSV.1.4.edit.vcf | wc -l
done

In [ ]:
# run in local terminal
for sample in 'KOLF2-ARID2-A2' 'KUCG3-C1' 'LNGPI1-C1' 'NCRM1-C6' 'NCRM5-C5' 'NN0003932-C3' 'NN0004297-C1' 'PGP1-C2'
do
svtk standardize ${sample}.diploidSV.1.4.edit.vcf ${sample}.diploidSV.standardize.vcf manta
done

In [207]:
#remove temp files
cmd4 = f'rm {WRKDIR2}/temp*diploidSV*.vcf'
!{cmd4}
#!echo ${cmd4}  # wrong syntax : can't combinate $ and {} in the !style

In [208]:
%ls -lth manta-output/


total 6553560
-rw-r--r--  1 pengl7  NIH\Domain Users   828K Sep 14 15:41 PGP1-C2.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   842K Sep 14 15:41 NN0004297-C1.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   714K Sep 14 15:41 NN0003932-C3.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   801K Sep 14 15:41 NCRM5-C5.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   827K Sep 14 15:41 NCRM1-C6.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   760K Sep 14 15:41 LNGPI1-C1.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   1.0M Sep 14 15:41 KUCG3-C1.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   860K Sep 14 15:41 KOLF2-ARID2-A2.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   2.5M Sep 14 15:36 PGP1-C2.diploidSV.1.4.edit.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   2.5M Sep 14 15:36 NN0004297-C1.diploidSV.1.4.edit.vcf
-rw-r--r--  1 pengl7  NIH\Domain Us

In [206]:
samples_list

['KOLF2-ARID2-A2',
 'KUCG3-C1',
 'LNGPI1-C1',
 'NCRM1-C6',
 'NCRM5-C5',
 'NN0003932-C3',
 'NN0004297-C1',
 'PGP1-C2']

In [209]:
##ran on local
##create a list file with path to the per sample standardize.vcf files

out_file_name = f'{WRKDIR2}/sv-manta.standardize.vcf.list'

# out_file = open(f'{out_file_name}', "w")
# for sample in samples_list:
    
#     direct = f'{WRKDIR}/{sample}.diploidSV.standardize.vcf'
#     print(f'{direct}', file=out_file)

with open(f'{out_file_name}', "w") as output:
    for sample in samples_list:
        direct = f'{WRKDIR2}/{sample}.diploidSV.standardize.vcf'
        output.write(direct + '\n')
      

In [213]:
%cat manta-output/sv-manta.standardize.vcf.list

/Users/pengl7/Downloads/WGS/SV_calling/manta-output/KOLF2-ARID2-A2.diploidSV.standardize.vcf
/Users/pengl7/Downloads/WGS/SV_calling/manta-output/KUCG3-C1.diploidSV.standardize.vcf
/Users/pengl7/Downloads/WGS/SV_calling/manta-output/LNGPI1-C1.diploidSV.standardize.vcf
/Users/pengl7/Downloads/WGS/SV_calling/manta-output/NCRM1-C6.diploidSV.standardize.vcf
/Users/pengl7/Downloads/WGS/SV_calling/manta-output/NCRM5-C5.diploidSV.standardize.vcf
/Users/pengl7/Downloads/WGS/SV_calling/manta-output/NN0003932-C3.diploidSV.standardize.vcf
/Users/pengl7/Downloads/WGS/SV_calling/manta-output/NN0004297-C1.diploidSV.standardize.vcf
/Users/pengl7/Downloads/WGS/SV_calling/manta-output/PGP1-C2.diploidSV.standardize.vcf


In [214]:
WRKDIR2

'/Users/pengl7/Downloads/WGS/SV_calling/manta-output'

In [215]:
print('run in terminal:')
print("")
cmd2 = f'svtk vcfcluster -t INS,DEL,DUP,INV,BND {WRKDIR2}/sv-manta.standardize.vcf.list sv-manta.standardize.cluster.vcf'
print(cmd2)

run in terminal:

svtk vcfcluster -t INS,DEL,DUP,INV,BND /Users/pengl7/Downloads/WGS/SV_calling/manta-output/sv-manta.standardize.vcf.list sv-manta.standardize.cluster.vcf


[W::vcf_parse] INFO/END=7147042 is smaller than POS at 1:10401969

In [216]:
%ls -lth manta-output/

total 6559784
-rw-r--r--  1 pengl7  NIH\Domain Users   2.3M Sep 17 16:10 sv-manta.standardize.cluster.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   710B Sep 17 16:06 sv-manta.standardize.vcf.list
-rw-r--r--  1 pengl7  NIH\Domain Users   828K Sep 14 15:41 PGP1-C2.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   842K Sep 14 15:41 NN0004297-C1.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   714K Sep 14 15:41 NN0003932-C3.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   801K Sep 14 15:41 NCRM5-C5.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   827K Sep 14 15:41 NCRM1-C6.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   760K Sep 14 15:41 LNGPI1-C1.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   1.0M Sep 14 15:41 KUCG3-C1.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users   860K Sep 14 15:41 KOLF2-ARID2-A2.diploidSV.standardize.vcf
-rw-r--r--  1 pengl7  NIH\Domain Users 

In [267]:
%%bash
cd manta-output/
gsutil -m cp sv.md5 gs://singlecellindi/WGS/Jax/IlluminaWGS/structural-variants-after-svtk/

Copying file://sv.md5 [Content-Type=application/octet-stream]...
/ [1/1 files][  787.0 B/  787.0 B] 100% Done                                    
Operation completed over 1 objects/787.0 B.                                      


In [436]:
# copy the data to the singlecellindi bucket and follow the folder structure of the last one analyzed by Anni
# folder of structure-varaints containes the vcf files after svtk processing and merge
# folder of sv-manta contains the vcf after manta analysis
!gsutil ls

gs://singlecellindi/
gs://test-7cee72c0e768/
gs://transfer_27may/
